In [1]:
import numpy as np
import pandas as pd
import torch
import string
import re
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from sklearn.preprocessing import LabelEncoder
from torchtext.vocab import build_vocab_from_iterator

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
training_df = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv', header=None)
validation_df = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv', header=None)
training_df.shape, validation_df.shape

((74682, 4), (1000, 4))

In [4]:
print(f'Shape before dropping nulls {training_df.shape}')
training_df = training_df.dropna()
validation_df = validation_df.dropna()
print(f'Shape after dropping nulls {training_df.shape}')

Shape before dropping nulls (74682, 4)
Shape after dropping nulls (73996, 4)


In [5]:
validation_df.loc[:, 3] = validation_df.loc[:, 3].str.replace(re.escape(string.punctuation), ' ', regex=True)
validation_df.loc[:, 3] = validation_df.loc[:, 3].str.replace(',', ' ', regex=True)

validation_df.loc[:, 3] = validation_df.loc[:, 3].str.replace(re.escape(string.punctuation), ' ', regex=True)
validation_df.loc[:, 3] = validation_df.loc[:, 3].str.replace(',', ' ', regex=True)

In [6]:
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data):
    for text in data:
        yield tokenizer(text)

In [7]:
training_texts = training_df[3].tolist()
val_texts = validation_df[3].tolist()

vocab = build_vocab_from_iterator(yield_tokens(training_texts+val_texts), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

text_pipeline = lambda x: vocab(tokenizer(x))
(text_pipeline('im happy'))



[313, 189]

In [8]:
int_training_texts = list(map(text_pipeline, training_texts))
int_val_texts = list(map(text_pipeline, val_texts))

print(list(map(len, [int_training_texts, int_val_texts])))

[73996, 1000]


In [9]:
le = LabelEncoder()
le.fit(training_df[2])

training_df['Labels']  = le.transform(training_df[2])
training_output_y = training_df['Labels'].tolist()

validation_df['Labels']  = le.transform(validation_df[2])
validation_output_y = validation_df['Labels'].tolist()

In [ ]:
def get_input_output(training_df, validation_df):
    training_texts = training_df[3].tolist()
    val_texts = validation_df[3].tolist()

    tokenizer = get_tokenizer('basic_english')

    training_tokenized = [tokenizer(text) for text in training_texts]
    validation_tokenized = [tokenizer(text) for text in val_texts]
                       
    training_flattened_list = [token for tokens in training_tokenized for token in tokens]
    validation_flattened_list = [token for tokens in validation_tokenized for token in tokens]

    vocab = sorted(list(set(training_flattened_list)) + list(set(validation_flattened_list)))
    word_to_id = {word:i+1 for i, word in enumerate(vocab)}
    id_to_word = {i+1:word for i, word in enumerate(vocab)}

    le = LabelEncoder()
    le.fit(training_df[2])
    
    training_df['Labels']  = le.transform(training_df[2])
    training_output_y = training_df['Labels'].tolist()
    
    validation_df['Labels']  = le.transform(validation_df[2])
    validation_output_y = validation_df['Labels'].tolist()
    
    print(list(map(len, [training_tokenized, validation_tokenized, training_output_y, validation_output_y])))
    
    return training_tokenized, validation_tokenized, training_output_y, validation_output_y, word_to_id
                       
train_tokenized_texts, val_tokenized_texts, train_output_y, val_output_y, word_to_id = get_input_output(training_df, validation_df)

In [ ]:
# # Read glove embeddings
# glove_path = '/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt'

# with open(glove_path, 'r') as file:
#      lines = file.readlines()

# print(len(lines))

In [ ]:
# word_to_vec = dict()
# for line in lines:
#     word_and_vec = line.split(' ', maxsplit=1)
#     word, vec = word_and_vec[0], word_and_vec[1]
#     vec_array = np.fromstring(vec, sep=' ').astype('float32')
#     word_to_vec[word] = vec_array


In [10]:
def padded_and_convert(tokens, output):
#     encode_text = lambda x: [word_to_id[_] for _ in x]

#     encoded_inputs = list(map(encode_text, tokens))
    padded = pad_sequence(list(map(torch.tensor, tokens)), batch_first=True)
    output_y = torch.tensor(output, dtype=torch.float32).unsqueeze(-1)
    output_y = output_y.type(torch.LongTensor)
    print(padded.shape, output_y.shape)
    
    return padded, output_y

train_padded, train_y = padded_and_convert(int_training_texts, training_output_y)
val_padded, val_y = padded_and_convert(int_val_texts, validation_output_y )

max_sequence_length = max(max(len(seq) for seq in train_padded), max(len(seq) for seq in val_padded))
print(max_sequence_length)

train_padded = pad_sequence([torch.cat([seq, torch.zeros(max_sequence_length - len(seq))]) for seq in train_padded], batch_first=True)
val_padded = pad_sequence([torch.cat([seq, torch.zeros(max_sequence_length - len(seq))]) for seq in val_padded], batch_first=True)
print(train_padded.shape, val_padded.shape)

torch.Size([73996, 311]) torch.Size([73996, 1])
torch.Size([1000, 73]) torch.Size([1000, 1])
311
torch.Size([73996, 311]) torch.Size([1000, 311])


In [11]:
def get_masks(padded):
    B, T = padded.shape
    mask = torch.eq(padded, 0).to(torch.float32)
    mask = mask * -1e9
    masked_reshape = mask.reshape(B, 1, T)
    return masked_reshape

train_mask = get_masks(train_padded)
val_mask = get_masks(val_padded)
print(train_mask.shape, val_mask.shape)

torch.Size([73996, 1, 311]) torch.Size([1000, 1, 311])


In [12]:
def get_angle(timesteps, dim):
    k = np.arange(dim)[np.newaxis, :]
    i = k // 2

    positions = np.arange(timesteps)[:, np.newaxis]
    angles = positions / (10000 ** (2*i/dim))

    return angles

def get_positional_embeddings(angles):
    angles[:, 0::2] = np.sin(angles[:, 0::2])
    angles[:, 1::2] = np.cos(angles[:, 1::2])

    return torch.tensor(angles, dtype=torch.float32)
n_embed = 64
timesteps = train_padded.shape[-1]
print(timesteps)


311


In [13]:
class Embedding(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        self.embedding_layer = nn.Embedding(n_vocab, n_embed)
        
    def forward(self, x):
        return self.embedding_layer(x)

In [14]:
class Head(nn.Module):
    def __init__(self, head_size=16):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size)
        self.key = nn.Linear(n_embed, head_size)
        self.value = nn.Linear(n_embed, head_size)
        
    def forward(self, x, mask):
        B, T, C = x.shape
        
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)
        
        wei = query @ key.transpose(-2, -1)
        
        if mask is not None:
            wei = wei + mask
        
        wei = F.softmax(wei, dim=-1)
        out = wei @ value # (B, T, head_size)
        
        return out

In [15]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, n_embed, timesteps, head_size, output):
        super().__init__() # What happens if I pass the class name in super?
        self.embedding = Embedding(n_vocab, n_embed)
        self.sa = Head(head_size)
        self.inter1_layer = nn.Linear(head_size, timesteps)
        self.output = nn.Linear(timesteps**2, output)
        
    def forward(self, x, mask, positional_encoding):
        B, T = x.shape # validation shape --> (1000, 73)
#         print(self.embedding(x).shape, positional_encoding.shape)
        embedding = self.embedding(x) + positional_encoding # (B, timesteps, n_embed) # validation: (1000, 73, 64)
        sa_out = self.sa(embedding, mask) # (B, timesteps, head_size) # validation: (1000, 73, 16)
        inter1 = self.inter1_layer(sa_out) # (B, timesteps, head_size) @ (head_size, timesteps) --> (B, timesteps, timesteps)
        output = self.output(inter1.view(B, -1))
#         output = F.softmax(output, dim=-1)
        
        return output

In [ ]:
# split = 0.8
# train_size = int(split * padded.shape[0])

# x_train, x_val, y_train, y_val = padded[:train_size], padded[train_size:], output_y[:train_size], output_y[train_size:]
# x_train, x_val, y_train, y_val = x_train.to(device), x_val.to(device), y_train.to(device), y_val.to(device)
# mask_train, mask_val = masked_reshape[:train_size], masked_reshape[train_size:]
# mask_train, mask_val = mask_train.to(device), mask_val.to(device)
# list(map(lambda x: x.shape, [x_train, x_val, y_train, y_val, mask_train, mask_val]))

In [16]:
def train_epoch(x_batch, mask, y_batch, positional_encoding):
    optimizer.zero_grad()
    output = model(x_batch, mask, positional_encoding)
    
    loss = loss_function(output, y_batch.view(-1))

    loss.backward()
    optimizer.step()
    
    return loss

In [17]:
train_padded, train_y = train_padded.to(device).long(), train_y.to(device)
val_padded, val_y = val_padded.to(device).long(), val_y.to(device) 
train_mask, val_mask = train_mask.to(device), val_mask.to(device)
train_positional_encoding = get_positional_embeddings(get_angle(timesteps, n_embed)).to(device)
val_positional_encoding = get_positional_embeddings(get_angle(val_padded.shape[-1], n_embed)).to(device)
print('Shape of encodings are', train_positional_encoding.shape, val_positional_encoding.shape)

n_embed = 64
timesteps = train_padded.shape[-1]
model = Encoder(vocab.get_itos().__len__(), n_embed, timesteps, head_size=16, output=4)
batch_size = 64
batch_per_epoch = train_padded.shape[0] // batch_size

# loss_function = nn.BCELoss()
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Set the device (CPU or GPU)
model.to(device)

Shape of encodings are torch.Size([311, 64]) torch.Size([311, 64])


Encoder(
  (embedding): Embedding(
    (embedding_layer): Embedding(43587, 64)
  )
  (sa): Head(
    (query): Linear(in_features=64, out_features=16, bias=True)
    (key): Linear(in_features=64, out_features=16, bias=True)
    (value): Linear(in_features=64, out_features=16, bias=True)
  )
  (inter1_layer): Linear(in_features=16, out_features=311, bias=True)
  (output): Linear(in_features=96721, out_features=4, bias=True)
)

In [19]:

# model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)
# output_y = output_y.type(torch.LongTensor)
# inputs, targets, masked_reshape = padded.to(device), output_y.to(device), masked_reshape.to(device)

for epoch in range(10, 20):
    train_loss, val_loss = 0, 0
    for i in range(batch_per_epoch):
        start = i * batch_size
        x_batch, y_batch, mask = train_padded[start:start+batch_size], train_y[start:start+batch_size], train_mask[start:start+batch_size]
#         x_batch, y_batch, mask = x_batch.to(device).long(), y_batch.to(device).long(), mask.to(device).long()

        model.train(True)
        loss = train_epoch(x_batch, mask, y_batch, train_positional_encoding)
        train_loss += loss
        
    print(f'Epoch {epoch} Loss: {train_loss / (i+1)}')   
    model.eval()
    with torch.no_grad():
        output_val = model(val_padded, val_mask, val_positional_encoding)
        loss_val = loss_function(output_val, val_y.view(-1))
        print(f'Epoch {epoch} Val loss: {loss_val}')
        
    print()

Epoch 10 Loss: 0.3506045639514923
Epoch 10 Val loss: 0.8817847371101379

Epoch 11 Loss: 0.3105768859386444
Epoch 11 Val loss: 0.868789553642273

Epoch 12 Loss: 0.27753058075904846
Epoch 12 Val loss: 1.0192766189575195

Epoch 13 Loss: 0.25311312079429626
Epoch 13 Val loss: 1.0108013153076172

Epoch 14 Loss: 0.2312641143798828
Epoch 14 Val loss: 0.9148634076118469

Epoch 15 Loss: 0.22323556244373322
Epoch 15 Val loss: 0.9734217524528503

Epoch 16 Loss: 0.1985422670841217
Epoch 16 Val loss: 0.9784271121025085

Epoch 17 Loss: 0.18999694287776947
Epoch 17 Val loss: 0.9342783689498901

Epoch 18 Loss: 0.1754094809293747
Epoch 18 Val loss: 0.9949996471405029

Epoch 19 Loss: 0.17757613956928253
Epoch 19 Val loss: 1.2491557598114014



In [ ]:
nested_list = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flattened_list = sum(nested_list, [])
print(flattened_list)
